# In this notebook, we're goint to cover some of the most fundamental concepts of tensors using TensorFlow

More specifically, we're going to cover:
* Introduction to tensors
* Getting information from tensors
* Manipulating tensors
* Tensors & NumPy
* Using @tf.function (a way to speed up your regular Python function)
* Using GPUs with Tensorflow (or TPUs)
* Exercises to try for yourself!

## Introduction to Tensors

In [1]:
# Import TensorFlow
import tensorflow as tf
print(tf.__version__)

2.19.0


In [2]:
# Create tensors with tf.constant()
scalar = tf.constant(7) #tf.constant creates a constant tensor from a tensor-like object
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [ ]:
# Check the number of dimensions of a tensor (ndim stands for number of dimensions)
scalar.ndim

0

In [4]:
# Create a vector
vector = tf.constant([10, 10]) #Note that the shape changes when you enter a list with more than one number to a tensor
vector #Also a numpy attribute appears when there's more than one number in the tensor

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [5]:
# Check the dimension of our vector
vector.ndim

1

In [6]:
# Create a matrix (has more than 1 dimension)
matrix = tf.constant([[10, 7], [7, 10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [7]:
matrix.ndim #Notice that the number of elements in a tensor's shape relates to its ndim

2

In [8]:
# Creat another matrix
another_matrix = tf.constant([[10., 7.], [3., 2.], [9., 9.]], dtype = tf.float16)
#Specify the data type with the dtype parameter; by default, the data type is int32
another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 3.,  2.],
       [ 9.,  9.]], dtype=float16)>

In [9]:
# What's the number dimensions of another_matrix?
another_matrix.ndim #Despite the shape being different, ndim is still 2; that's because ndim is the total number of elements in its shape attribute

2

In [10]:
# Let's create a tensor
tensor = tf.constant([[[1, 2, 3,], [4, 5, 6]], [[7, 8, 9], [10, 11, 12]], [[13, 14, 15], [16, 17, 18]]])
tensor #Now there's three elements in the tensor's shape; can you guess what its ndim will be?

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

In [11]:
tensor.ndim

3

What we've created so far:

* Scalar - A single number
* Vector - A number with direction (e.g. wind speed and direction)
* Matrix - A 2-dimensional array of numbers
* Tensor - An n-dimensional array of numbers (where n can be any number, a 0-dimensional tensor is a scalar, a 1-dimensonal tenors is a vector)

### Creating Tensors With 'tf.Variable':

In [12]:
# Create the same tensor with tv.Variable() as above
changeable_tensor = tf.Variable([10, 7])
unchangeable_tensor = tf.constant([10, 7])
changeable_tensor, unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [13]:
# Let's try to change one of the elements in our changeable tensor
changeable_tensor[0] = 7 #Variable assignment is not the way to modify tensor elements
changeable_tensor

TypeError: 'ResourceVariable' object does not support item assignment

In [14]:
# How about we try .assign()
changeable_tensor[0].assign(7)
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7], dtype=int32)>

In [15]:
# Let's try to change our unchangeable tensor
unchangeable_tensor[0].assign(7) #You have to know ahead of time whether or not you want a tensor to be mutable
unchangeable_tensor #Often times, the decision between constant and variable tensors will be made behind the scenes when Tensorflow creates tensors for your neural networks

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'assign'

🔑Note: Rarely in practice will you need to decide whether to use tf.constant or tf.variable to create tensors, as TensorFlow does this for you. However, if in doubt, use tf.constant and change it later if needed.

### Creating Random Tensors:

Random tensors are tensors of some arbitrary size which contain random numbers.

In [16]:
# Create two random (but the same) tensors
random_1 = tf.random.Generator.from_seed(42) # Set seed for reproducibility
random_1 = random_1.normal(shape = (3, 2)) # Outputs random values from a normal distribution
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape = (3, 2))

#Are they equal?
random_1, random_2, random_1 == random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

In [17]:
# Using a different random seed
random_1 = tf.random.Generator.from_seed(7)
random_1 = random_1.normal(shape = (3, 2))
random_2 = tf.random.Generator.from_seed(7)
random_2 = random_2.normal(shape = (3, 2))
random_1, random_2, random_1 == random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-1.3240396 ,  0.28785667],
        [-0.8757901 , -0.08857018],
        [ 0.69211644,  0.84215707]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-1.3240396 ,  0.28785667],
        [-0.8757901 , -0.08857018],
        [ 0.69211644,  0.84215707]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

### Shuffle The Order Of Elements In A Tensor:

In [18]:
# Shuffle a tensor (valuable for when you want to shuffle your data so the inherent order doesn't affect learning)
not_shuffled = tf.constant([[10, 7], [3, 4], [2, 5]])

# Shuffle our non-shuffled tensor
tf.random.shuffle(not_shuffled)

#Note: Shuffling a tensor via tf.random.shuffle only shuffles it along its first dimension

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 3,  4],
       [10,  7],
       [ 2,  5]], dtype=int32)>

In [19]:
tf.random.shuffle(not_shuffled) #Every time you run this cell, the order changes

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 2,  5],
       [ 3,  4],
       [10,  7]], dtype=int32)>

In [20]:
tf.random.shuffle(not_shuffled, seed = 42) #Using seed still yields different results every time

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 2,  5],
       [ 3,  4],
       [10,  7]], dtype=int32)>

In [21]:
#Operations that rely on a random seed actually derive it from two seeds: the global and operation=level seeds
tf.random.set_seed(42) #This sets the global seed
tf.random.shuffle(not_shuffled, seed = 42) #The seed parameter sets the operation-level seed

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

🛠️ **Exercise** Read through TensorFlow documentation on random seed generation: https://www.tensorflow.org/api_docs/python/tf/random/set_seed and practice writing 5 random tensors and shuffle them.

It looks like if we want our shuffled tensors to be in the same order, we've got to use the global level random seed as well as the operation level random seed:

> Rule 4: "If both the global and the operation seed are set: Both seeds are used in conjunction to determine the random sequence."

### Other Ways To Make Tensors:

In [22]:
# Create a tensor of all ones:
tf.ones([10, 7]) #tf.ones utilizes numpy's ones method to take in a shape to create a tensor of all 1's

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [23]:
# Create a tensor of all zeroes:
tf.zeros(shape = (3, 4)) #Same as above, except for a tensor of all 0's
#Can either pass in a list or a tuple for the shape parameter

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

### Turn NumPy Arrays Into Tensors:

The main difference between NumPy arrays and TensorFlow is that tensors can be run on a GPU (much faster for numerical computing).

In [24]:
# You can also turn NumPy arrays into tensors
import numpy as np
numpy_A = np.arange(1, 25, dtype = np.int32) # create a NumPy array between 1 and 25
numpy_A

# X = tf.constant(some_matrix) # capital for matrix or tensor
# y = tf.constant(vector) # non-capital for vector
# Similar notation to X and y using scikit-learn

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)

In [25]:
A = tf.constant(numpy_A) #Numpy arrays can be directly passed into any tensor generation method
A

<tf.Tensor: shape=(24,), dtype=int32, numpy=
array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)>

In [26]:
A = tf.constant(numpy_A, shape = (2, 3, 4)) #You can change the shape of the object being passed in when creating a tensor if the result has the same number of total elements
B = tf.constant(numpy_A)
A, B

(<tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
 array([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]],
 
        [[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]]], dtype=int32)>,
 <tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24], dtype=int32)>)

In [27]:
2 * 3 * 4

24

In [28]:
A = tf.constant(numpy_A, shape = (2, 3, 5)) #You cannot convert an array into a tensor of a different shape
A

TypeError: Eager execution of tf.constant with unsupported shape. Tensor [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24] (converted from [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]) has 24 elements, but got `shape` (2, 3, 5) with 30 elements).

In [29]:
A = tf.constant(numpy_A, shape = (3, 8)) #The number of dimensions don't matter; only the total number of elements does
A

<tf.Tensor: shape=(3, 8), dtype=int32, numpy=
array([[ 1,  2,  3,  4,  5,  6,  7,  8],
       [ 9, 10, 11, 12, 13, 14, 15, 16],
       [17, 18, 19, 20, 21, 22, 23, 24]], dtype=int32)>

### Getting Information From Tensors:

When dealing with tensors you probably want to be a ware of the following attributes:
* Shape - The length (number of elements) of each of the dimensions of a tensor.
* Rank - The number of tensor dimensions. A scalar has rank 0, a vector has rank 1, a matrix has rank 2, and a tensor has rank n.
* Axis Or Dimension - A particular dimension of a tensor.
* Size - The total number of items in the tensor.

In [30]:
# Create a rank 4 tensor (4 dimensions)
rank_4_tensor = tf.zeros([2, 3, 4, 5])
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]], dtype=float32)>

In [31]:
rank_4_tensor[0]

<tf.Tensor: shape=(3, 4, 5), dtype=float32, numpy=
array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]], dtype=float32)>

In [32]:
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor)

(TensorShape([2, 3, 4, 5]), 4, <tf.Tensor: shape=(), dtype=int32, numpy=120>)

In [33]:
2 * 3 * 4 * 5

120

In [34]:
# Get various attributes of our tensor:
print('Datatype of every element:', rank_4_tensor.dtype)
print('Number of dimensions (rank):', rank_4_tensor.ndim)
print('Shape of tensor:', rank_4_tensor.shape)
print('Elements along the 0 axis:', rank_4_tensor.shape[0])
print('Elements along the last axis:', rank_4_tensor.shape[-1])
print('Total number of elements in our tensor:', tf.size(rank_4_tensor))
print('Total number of elements in our tensor:', tf.size(rank_4_tensor).numpy())

Datatype of every element: <dtype: 'float32'>
Number of dimensions (rank): 4
Shape of tensor: (2, 3, 4, 5)
Elements along the 0 axis: 2
Elements along the last axis: 5
Total number of elements in our tensor: tf.Tensor(120, shape=(), dtype=int32)
Total number of elements in our tensor: 120


### Indexing Tensors:

Tensors can be indexed just like Python lists.

In [35]:
some_list = [1, 2, 3, 4]
some_list[:2]

[1, 2]

In [36]:
# Get the first 2 elements of each dimension:
rank_4_tensor[:2, :2, :2, :2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]], dtype=float32)>

In [37]:
some_list[:1]

[1]

In [38]:
# Get the first element each dimension from each index except for the final one:
rank_4_tensor[:1, :1, :1, :]

<tf.Tensor: shape=(1, 1, 1, 5), dtype=float32, numpy=array([[[[0., 0., 0., 0., 0.]]]], dtype=float32)>

In [39]:
# Same as above, except with the second to last one instead:
rank_4_tensor[:1, :1, :, :1]

<tf.Tensor: shape=(1, 1, 4, 1), dtype=float32, numpy=
array([[[[0.],
         [0.],
         [0.],
         [0.]]]], dtype=float32)>

In [40]:
# Create a rank 2 tensor (2 dimensions):
rank_2_tensor = tf.constant([[10, 7], [3, 4]])
rank_2_tensor.shape, rank_2_tensor.ndim

(TensorShape([2, 2]), 2)

In [41]:
some_list, some_list[-1]

([1, 2, 3, 4], 4)

In [42]:
# Get the last item of each row of our rank 2 tensor:
rank_2_tensor[:, -1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([7, 4], dtype=int32)>

In [43]:
# Add in extra dimension to our rank 2 tensor without changing the numbers:
rank_3_tensor = rank_2_tensor[..., tf.newaxis]
rank_3_tensor
#Three dots means on every axis before a certain one

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]], dtype=int32)>

In [44]:
# Alternative to tf.newaxis:
tf.expand_dims(rank_2_tensor, axis = -1) #"-1" means expand the final axis

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]], dtype=int32)>

In [45]:
tf.expand_dims(rank_2_tensor, axis = 0) # Expand the 0-axis

<tf.Tensor: shape=(1, 2, 2), dtype=int32, numpy=
array([[[10,  7],
        [ 3,  4]]], dtype=int32)>

In [46]:
tf.expand_dims(rank_2_tensor, axis = 1)

<tf.Tensor: shape=(2, 1, 2), dtype=int32, numpy=
array([[[10,  7]],

       [[ 3,  4]]], dtype=int32)>

### Manipulating Tensors (Tensor Operations):

**Basic Operations**

'+', '-', '*', '/'

In [47]:
# You can add values to a tensor using the addition operator
tensor = tf.constant([[10, 7], [3, 4]])
tensor + 10 #Adds an amount to every element in a tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]], dtype=int32)>

In [48]:
# Original tensor is unchanged
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

In [49]:
tensor = tensor + 10 #Overwrite the tensor via reassignment
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]], dtype=int32)>

In [50]:
# Multiplication also works
tensor = tf.constant([[10, 7], [3, 4]])
tensor * 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]], dtype=int32)>

In [51]:
# Subtraction if you want
tensor - 10 #All of these operations are element-wise

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 0, -3],
       [-7, -6]], dtype=int32)>

In [52]:
# We can use the tensorflow built-in function too
tf.multiply(tensor, 10) #It's recommended to use the tensorflow functions when using GPUs or TPUs
#The tensorflow math functions are tf.add(), tf.subtract(), tf.multiply(), and tf.divide()

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]], dtype=int32)>

In [53]:
tensor #The original tensor is still unchanged

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

**Matrix Multiplication**

In machine learning, matrix multiplication is one of the most common tensor operations.

There are two rules our tensors (or matrices) need to fulful if we're going to matrix multiply them:

1. The inner dimensions must match
2. The resulting matrix has the shape of the outer dimensions

Ex: Multiplying a 3x3 matrix by a 3x2 matrix is legal because the inner dimensions (3x**3** and **3**x2) are the same, and this will result in a 3x2 matrix

In [54]:
# Matrix multiplication in tensorflow
print(tensor)
tf.matmul(tensor, tensor)

tf.Tensor(
[[10  7]
 [ 3  4]], shape=(2, 2), dtype=int32)


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

In [55]:
tensor, tensor

(<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[10,  7],
        [ 3,  4]], dtype=int32)>,
 <tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[10,  7],
        [ 3,  4]], dtype=int32)>)

In [56]:
tensor * tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  49],
       [  9,  16]], dtype=int32)>

In [57]:
# Matrix multiplication with Python operator "@"
tensor @ tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

In [58]:
tensor.shape # Pay attention to the shapes of your tensors you want to multiply!

TensorShape([2, 2])

In [59]:
# Create a tensor (3, 2) tensor:
X = tf.constant([[1, 2], [3, 4], [5, 6]])

# Create another (3, 2) tensor:
Y = tf.constant([[7, 8], [9, 10], [11, 12]])

X, Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]], dtype=int32)>)

In [60]:
# Try to matrix multiply tensors of same shape
X @ Y #This breaks rule 1 of the inner dimensions having to match

InvalidArgumentError: {{function_node __wrapped__MatMul_device_/job:localhost/replica:0/task:0/device:CPU:0}} Matrix size-incompatible: In[0]: [3,2], In[1]: [3,2] [Op:MatMul] name: 

In [61]:
tf.matmul(X, Y)

InvalidArgumentError: {{function_node __wrapped__MatMul_device_/job:localhost/replica:0/task:0/device:CPU:0}} Matrix size-incompatible: In[0]: [3,2], In[1]: [3,2] [Op:MatMul] name: 

In [62]:
Y

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 7,  8],
       [ 9, 10],
       [11, 12]], dtype=int32)>

In [63]:
# Let's change the shape of Y
tf.reshape(Y, shape = (2, 3))

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[ 7,  8,  9],
       [10, 11, 12]], dtype=int32)>

In [64]:
X.shape, tf.reshape(Y, shape = (2, 3)).shape

(TensorShape([3, 2]), TensorShape([2, 3]))

In [65]:
# Try to matrix multiply X by reshaped Y
X @ tf.reshape(Y, shape = (2, 3))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [66]:
tf.matmul(X, tf.reshape(Y, shape = (2, 3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [67]:
tf.reshape(X, shape = (2, 3)).shape, Y.shape

(TensorShape([2, 3]), TensorShape([3, 2]))

In [68]:
# Try to change the shape of X instead of Y
tf.matmul(tf.reshape(X, shape = (2, 3)), Y)
#Be aware that the resulting shape depends on the outer dimensions, so choose carefully which matrix to reshape!

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 58,  64],
       [139, 154]], dtype=int32)>

In [69]:
# Can do the same with transpose
X, tf.transpose(X), tf.reshape(X, shape = (2, 3))
#The difference between transpose and reshape is that transpose flips the axes while reshape just shuffles the tensor around into the shape you want

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 3, 5],
        [2, 4, 6]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 2, 3],
        [4, 5, 6]], dtype=int32)>)

In [70]:
# Try matrix multiplication with transpose rather than reshape
tf.matmul(tf.transpose(X), Y)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]], dtype=int32)>

📖 **Resource:** Info and example of matrix multiplication: https://www.mathisfun.com/algebra/matrix-multiplying.html

**The Dot Product**

Matrix multiplication is also referrecd to as the dot product.

You can perform matrix multiplication using:
* 'tf.matmul()'
* 'tf.tensordot()'
* '@'

In [71]:
X, Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]], dtype=int32)>)

In [72]:
# Perform the dot product on X and Y (requires X or Y to be transposed)
tf.tensordot(tf.transpose(X), Y, axes = 1) #Tensor contraction of a and b along specified axes and outer product

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]], dtype=int32)>

In [73]:
# Perform matrix multiplication between X and Y (transposed)
tf.matmul(X, tf.transpose(Y))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]], dtype=int32)>

In [74]:
# Perform matrix multiplication between X and Y (reshaped)
tf.matmul(X, tf.reshape(Y, shape = (2, 3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [75]:
# Check the values of Y, reshape Y and transposed Y
print('Normal Y:')
print(Y, '\n') # "\n" is for newline

print('Y reshaped to (2, 3):')
print(tf.reshape(Y, (2, 3)), '\n')

print('Y transposed:')
print(tf.transpose(Y))

Normal Y:
tf.Tensor(
[[ 7  8]
 [ 9 10]
 [11 12]], shape=(3, 2), dtype=int32) 

Y reshaped to (2, 3):
tf.Tensor(
[[ 7  8  9]
 [10 11 12]], shape=(2, 3), dtype=int32) 

Y transposed:
tf.Tensor(
[[ 7  9 11]
 [ 8 10 12]], shape=(2, 3), dtype=int32)


In [76]:
tf.matmul(X, tf.transpose(Y))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]], dtype=int32)>

Generally, when performing matrix multiplication on two tensors and one of the axes doesn't line up, you will transpose (rather than reshape) one of the tensores to get satisfy the multiplication rules.

### Changing The Datatype Of A Tensor:

In [77]:
tf.__version__

'2.19.0'

In [78]:
# Create a new tensor with default datatype (float32)
B = tf.constant([1.7, 7.4])
B, B.dtype

(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1.7, 7.4], dtype=float32)>,
 tf.float32)

In [79]:
C = tf.constant([7, 10]) #If your tensor doesn't have decimals in any of its values, it will default to int32 rather than float32
C, C.dtype

(<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 7, 10], dtype=int32)>,
 tf.int32)

In [80]:
# Change from float32 to float16 (reduced precision)
D = tf.cast(B, dtype = tf.float16)
D, D.dtype

(<tf.Tensor: shape=(2,), dtype=float16, numpy=array([1.7, 7.4], dtype=float16)>,
 tf.float16)

In [81]:
# Change from int32 to float32
E = tf.cast(C, dtype = tf.float32)
E, E.dtype

(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 7., 10.], dtype=float32)>,
 tf.float32)

In [82]:
E_float16 = tf.cast(E, dtype = tf.float16)
E_float16, E_float16.dtype

(<tf.Tensor: shape=(2,), dtype=float16, numpy=array([ 7., 10.], dtype=float16)>,
 tf.float16)

### Aggregating Tensors:

Aggregating tensors = condensing them from multiple values down to a smaller amount of values.

In [83]:
D = tf.constant([-7, -10])
D

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ -7, -10], dtype=int32)>

In [84]:
# Get the absolute values
tf.abs(D)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 7, 10], dtype=int32)>

Let's go through the following forms of aggregation:
* Get the minimum
* Get the maximum
* Get the mean of a tensor
* Get the sum of a tensor

In [85]:
# Creating a random tensor with values between 0 and 100 of size 50
E = tf.constant(np.random.randint(0, 100, size = 50)) #np.random.randint() returns random integers from 'low' (inclusive) to 'high' (exclusive)
E

<tf.Tensor: shape=(50,), dtype=int32, numpy=
array([76, 25, 13, 24, 37, 27, 35, 13, 79, 41, 48, 71, 89, 70, 66, 75, 70,
       17, 35,  7, 48,  8, 55, 90, 17, 45, 25, 13, 78, 77,  4, 20,  4,  4,
       35, 80, 64, 83, 42, 36, 26, 77, 72, 17, 72, 96, 71, 36, 45, 74],
      dtype=int32)>

In [86]:
tf.size(E), E.shape, E.ndim

(<tf.Tensor: shape=(), dtype=int32, numpy=50>, TensorShape([50]), 1)

In [87]:
# Find the minimum
tf.reduce_min(E) #TensorFlow tends to put 'reduce_' in its aggregation methods

<tf.Tensor: shape=(), dtype=int32, numpy=4>

In [88]:
# Find the maximum
tf.reduce_max(E)

<tf.Tensor: shape=(), dtype=int32, numpy=96>

In [89]:
# Find the mean
tf.reduce_mean(E)

<tf.Tensor: shape=(), dtype=int32, numpy=46>

In [90]:
# Find the sum
tf.reduce_sum(E)

<tf.Tensor: shape=(), dtype=int32, numpy=2332>

🛠️ **Exercise:** With what we've just learned, find the variance and standard deviation of our 'E' tensor using TensorFlow methods.

In [91]:
# Find the variance of our tensor
tf.reduce_var(E) #Variance is not found within the general tensorflow module

AttributeError: module 'tensorflow' has no attribute 'reduce_var'

In [92]:
tf.reduce_variance(E) #Variance is not found within the general tensorflow module

AttributeError: module 'tensorflow' has no attribute 'reduce_variance'

In [93]:
# To find the variance of our tensor, we need access to tensorflow_probability
import tensorflow_probability as tfp
tfp.stats.variance(E)

<tf.Tensor: shape=(), dtype=int32, numpy=749>

In [94]:
# Find the standard deviation
tf.reduce_std(E) #The reduce_std method is found under tf.math

AttributeError: module 'tensorflow' has no attribute 'reduce_std'

In [ ]:
tf.math.reduce_std(E) #Tensors must exclusively be in float format to have their standard deviations calculated, NOT int!

TypeError: Input must be either real or complex. Received integer type <dtype: 'int32'>.

In [96]:
tf.math.reduce_std(tf.cast(E, dtype = tf.float32))

<tf.Tensor: shape=(), dtype=float32, numpy=27.369150161743164>

In [97]:
# Find the variance of our E tensor
tf.math.reduce_variance(tf.cast(E, dtype = tf.float32)) #It turns out that variance is also found under the math submodule, but the input tensor also needs to be cast to float

<tf.Tensor: shape=(), dtype=float32, numpy=749.0703735351562>

## Find The Positional Maximum And Minimum:

What if you wanted to find the index within a tensor where the minimum or maximum value occurs?

In [98]:
# Create a new tensor for finding positional minimum and maximum
tf.random.set_seed(42)
F = tf.random.uniform(shape = [50])
F

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
       0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
       0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
       0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
       0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
       0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
       0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
       0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
       0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
       0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
      dtype=float32)>

In [99]:
# Find the positional maximum
tf.argmax(F) #Returns the index with the largest value across axes of a tensor

<tf.Tensor: shape=(), dtype=int64, numpy=42>

In [100]:
# Index on our largest value position
F[tf.argmax(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.967138409614563>

In [101]:
# Find the max value of F
tf.reduce_max(F)

<tf.Tensor: shape=(), dtype=float32, numpy=0.967138409614563>

In [102]:
# Check for equality
F[tf.argmax(F)] == tf.reduce_max(F)

<tf.Tensor: shape=(), dtype=bool, numpy=True>

In [103]:
# Find the positional minimum
tf.argmin(F) #Same as argmax, except with the smallest value instead

<tf.Tensor: shape=(), dtype=int64, numpy=16>

In [104]:
# Find the minimum using the positional minimum index
F[tf.argmin(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.009463667869567871>

### Squezing A Tensor (Removing All Single Dimensions):

In [105]:
# Create a tensor to get started
tf.random.set_seed(42)
G = tf.constant(tf.random.uniform(shape = [50]), shape = (1, 1, 1, 1, 50))
G

<tf.Tensor: shape=(1, 1, 1, 1, 50), dtype=float32, numpy=
array([[[[[0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
           0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
           0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
           0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
           0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
           0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
           0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
           0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
           0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
           0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043]]]]],
      dtype=float32)>

In [106]:
G.shape

TensorShape([1, 1, 1, 1, 50])

In [107]:
G_squeezed = tf.squeeze(G) #Removes dimensions of size 1 from the shape of a tensor
G_squeezed, G_squeezed.shape

(<tf.Tensor: shape=(50,), dtype=float32, numpy=
 array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
        0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
        0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
        0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
        0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
        0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
        0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
        0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
        0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
        0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
       dtype=float32)>,
 TensorShape([50]))

### One-Hot Encoding Tensors:

In [108]:
# Create a list of indeces:
some_list = [0, 1, 2, 3] # Could be red, green, blue, purple

# One hot encode our list of indeces:
tf.one_hot(some_list, depth = 4) #Please recall one-hot encoding from the scikit-learn machine learning course!

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)>

In [109]:
# Specify custom values for one hot encoding
tf.one_hot(some_list, depth = 4, on_value = 'Yo I love deep learning', off_value = 'I Play Pokemon Go Everyday')

<tf.Tensor: shape=(4, 4), dtype=string, numpy=
array([[b'Yo I love deep learning', b'I Play Pokemon Go Everyday',
        b'I Play Pokemon Go Everyday', b'I Play Pokemon Go Everyday'],
       [b'I Play Pokemon Go Everyday', b'Yo I love deep learning',
        b'I Play Pokemon Go Everyday', b'I Play Pokemon Go Everyday'],
       [b'I Play Pokemon Go Everyday', b'I Play Pokemon Go Everyday',
        b'Yo I love deep learning', b'I Play Pokemon Go Everyday'],
       [b'I Play Pokemon Go Everyday', b'I Play Pokemon Go Everyday',
        b'I Play Pokemon Go Everyday', b'Yo I love deep learning']],
      dtype=object)>

### Squaring, Log, Square Root:

In [110]:
# Create a new tensor
H = tf.range(1, 10)
H

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [111]:
#Square it
tf.square(H)

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([ 1,  4,  9, 16, 25, 36, 49, 64, 81], dtype=int32)>

In [112]:
# Find the squareroot
tf.sqrt(H) #Again, ints are illegal with certain math operations

InvalidArgumentError: Value for attr 'T' of int32 is not in the list of allowed values: bfloat16, half, float, double, complex64, complex128
	; NodeDef: {{node Sqrt}}; Op<name=Sqrt; signature=x:T -> y:T; attr=T:type,allowed=[DT_BFLOAT16, DT_HALF, DT_FLOAT, DT_DOUBLE, DT_COMPLEX64, DT_COMPLEX128]> [Op:Sqrt] name: 

In [113]:
# Find the squareroot
tf.sqrt(tf.cast(H, dtype = tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([1.       , 1.4142135, 1.7320508, 2.       , 2.236068 , 2.4494898,
       2.6457512, 2.828427 , 3.       ], dtype=float32)>

In [114]:
# Find the log
tf.log(H) #It's in the math submodule

AttributeError: module 'tensorflow' has no attribute 'log'

In [115]:
# Find the log
tf.math.log(H) #How did you know it was going to require type casting?

InvalidArgumentError: Value for attr 'T' of int32 is not in the list of allowed values: bfloat16, half, float, double, complex64, complex128
	; NodeDef: {{node Log}}; Op<name=Log; signature=x:T -> y:T; attr=T:type,allowed=[DT_BFLOAT16, DT_HALF, DT_FLOAT, DT_DOUBLE, DT_COMPLEX64, DT_COMPLEX128]> [Op:Log] name: 

In [116]:
# Find the log
tf.math.log(tf.cast(H, dtype = tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([0.       , 0.6931472, 1.0986123, 1.3862944, 1.609438 , 1.7917595,
       1.9459102, 2.0794415, 2.1972246], dtype=float32)>

### Tensors And Numpy:

TensorFlow interacts beautifly with Numpy arrays

🔑**Note:** One of the main differences between a TensorFlow tensor and a NumPy array is that a TensorFlow tensor can be run on a GPU or TPU (for faster numerical processing).

In [117]:
# Create a tensor directly from a NumPy array
J = tf.constant(np.array([3., 7., 10.]))
J

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 3.,  7., 10.])>

In [118]:
# Convert our tensor back to a NumPy array
np.array(J), type(np.array(J))

(array([ 3.,  7., 10.]), numpy.ndarray)

In [119]:
# Convert tensor J to a NumPy array
J.numpy(), type(J.numpy())

(array([ 3.,  7., 10.]), numpy.ndarray)

In [120]:
J = tf.constant([3.])
J.numpy()[0]

np.float32(3.0)

In [ ]:
# The default types of each are slightly different
numpy_J = tf.constant(np.array([3., 7., 10.]))
tensor_J = tf.constant([3., 7., 10.])
#Check the datatypes of each
numpy_J.dtype, tensor_J.dtype

(tf.float64, tf.float32)

### Finding Access To GPUs:

In [122]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [123]:
tf.config.list_physical_devices("GPU") #Unforutnately, GPU support for Tensorflow on Windows has ended as of version 2.11!

[]

In [124]:
!nvidia-smi

Sat Apr 19 16:55:36 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 538.92                 Driver Version: 538.92       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1650      WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   62C    P8               4W /  35W |    805MiB /  4096MiB |     30%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

🔑 **Note** If you have access to a CUDA-enabled GPU, TensorFlow will automatically use it whenever possible.

⚠️ **Update**: This is no longer true, as support for GPUs on Windows has ended as of Tensorflow 2.11